In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 


In [2]:
import geopandas
from geopandas import GeoDataFrame
from shapely.geometry import Point
import pyproj
from pyproj import Proj, transform

In [3]:
df = pd.read_csv('../data/LADOT_Metered_Parking_Inventory___Policies.csv')

In [4]:
df.head(1)

,SpaceID,BlockFace,MeterType,RateType,RateRange,MeteredTimeLimit,ParkingPolicy,StreetCleaning,LatLng
0,VS336,4900 SUNSET BL,Single-Space,FLAT,$1.00,2HR,"TANS 4P-7P Mon-Fri | PKG 8A-4P Mon-Fri, 8A-8P Sat",NaN,"(34.098024, -118.296664)"


In [5]:
df.shape

(33989, 9)

In [6]:
df['Latitude'] = df['LatLng'].str.replace("(", "").str.split().str[0]
df['Longitude'] = df['LatLng'].str.replace(")","").str.split().str[1]
df["Latitude"] = df['Latitude'].str[:-1].astype(float)
df['Longitude'] = df['Longitude'].astype(float)

In [7]:
import gmaps


gmaps.configure(api_key='') # Fill in with your API key



locations = df[["Latitude","Longitude"]]
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations))
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
mapbox_access_token = 'pk.eyJ1IjoiYnJlZXplMDk0IiwiYSI6ImNrODJvM2szZzA4ODMzbGxxNXFlZmU5cjQifQ.0V6bOGL5yUPBX7i_A5Gm-g'
import json
import geopandas 
from geopandas import GeoDataFrame
from shapely.geometry import Point, Polygon

In [9]:
#"Latitude","Longitude"
geometry = [Point(xy) for xy in zip(df['Longitude'], df['Latitude'])]

# Coordinate reference system : WGS84
crs = {'init': 'epsg:4326'}

# Creating a Geographic data frame 
gdf = geopandas.GeoDataFrame(df, crs=crs, geometry=geometry)

In [10]:
gdf.head(1)

,SpaceID,BlockFace,MeterType,RateType,RateRange,MeteredTimeLimit,ParkingPolicy,StreetCleaning,LatLng,Latitude,Longitude,geometry
0,VS336,4900 SUNSET BL,Single-Space,FLAT,$1.00,2HR,"TANS 4P-7P Mon-Fri | PKG 8A-4P Mon-Fri, 8A-8P Sat",NaN,"(34.098024, -118.296664)",34.098024,-118.296664,POINT (-118.29666 34.09802)


In [11]:
from_json = gdf.to_json()
geoJSON = json.loads(from_json)

In [12]:
geoJSON

{'type': 'FeatureCollection',
 'features': [{'id': '0',
   'type': 'Feature',
   'properties': {'BlockFace': '4900 SUNSET BL',
    'LatLng': '(34.098024, -118.296664)',
    'Latitude': 34.098024,
    'Longitude': -118.296664,
    'MeterType': 'Single-Space',
    'MeteredTimeLimit': '2HR',
    'ParkingPolicy': 'TANS 4P-7P Mon-Fri | PKG 8A-4P Mon-Fri, 8A-8P Sat',
    'RateRange': '$1.00',
    'RateType': 'FLAT',
    'SpaceID': 'VS336',
    'StreetCleaning': None},
   'geometry': {'type': 'Point', 'coordinates': [-118.296664, 34.098024]}},
  {'id': '1',
   'type': 'Feature',
   'properties': {'BlockFace': '700 E PICO BLVD',
    'LatLng': '(34.032054, -118.252521)',
    'Latitude': 34.032054,
    'Longitude': -118.252521,
    'MeterType': 'Single-Space',
    'MeteredTimeLimit': '2HR',
    'ParkingPolicy': '8A-6P Mon-Sat',
    'RateRange': '$0.5 - $1',
    'RateType': 'TOD',
    'SpaceID': 'ED396',
    'StreetCleaning': None},
   'geometry': {'type': 'Point', 'coordinates': [-118.252521, 34

In [13]:
pts=[]#list of points defining boundaries of polygons
for  feature in geoJSON['features']:
    if feature['geometry']['type']=='Point':
        pts.extend(feature['geometry']['coordinates'])    
        #pts.append([None, None])#mark the end of a polygon   

    elif feature['geometry']['type']=='MultiPolygon':
        for polyg in feature['geometry']['coordinates']:
            pts.extend(polyg[0])
            pts.append([None, None])#end of polygon
    else: raise ValueError("geometry type irrelevant for map") 

In [14]:
X = []
Y = []
for i in range(len(pts)):        
    if i % 2 == 0:
        X.append(pts[i])
    else:
        Y.append(pts[i])
        

In [16]:
X[:10]

[-118.296664,
 -118.252521,
 -118.264675,
 -118.274463,
 -118.44891,
 -118.344672,
 -118.27393,
 -118.266015,
 -118.369344,
 -118.445724]

In [18]:
data = [
go.Scattermapbox(
    lat=Y,
    lon=X,
    mode='markers',
    marker=dict(size= 3,color = 'red',opacity = .8))
    ]

layout = go.Layout(autosize=True,
                   mapbox = dict(center= dict(lat=33,     #Set center at Flataklocken
                                 lon=-118),        
                                 accesstoken=mapbox_access_token,
                                 zoom=4,
                                 style='light',
                               ),
                   
                    width=700,
                    height=600,
                    title = 'LA')


fig = go.Figure(data=data, layout=layout)
plot(fig, filename='LA Mapbox')

'LA Mapbox.html'

In [42]:
def get_polygon(lons, lats, color='blue'):
    if len(lons) != len(lats):
        raise ValueError('the legth of longitude list  must coincide with that of latitude')
    geoJSON = {"type": "FeatureCollection"}
    geoJSON['features'] = []
    coords = []
    for lon, lat in zip(X, Y): 
        coords.append((lon, lat))   
    coords.append((lons[0], lats[0]))  #close the polygon  
    geoJSON['features'].append({ "type": "Feature",
                               "geometry": {"type": "Point",
                                            "coordinates": [coords] }})
    layer=dict(sourcetype = 'geojson',
             source =geoJSON,
             below='',  
             type = 'fill',   
             color = color)
    return layer

In [48]:
mylayers =[]
mylayers.append(get_polygon(lons=[33,34,34,33], lats=[-118,-118,-119,-119],  color='gold'))
fig.layout.update(mapbox_layers =mylayers);